# World University Rankings 2025

**Authors:**  
Nguyen Ngo, Josh Ilko  
Penn State University  
DS 220 – Spring 2025  
May 5, 2025

In this data analysis, we will answer exploratory questions about approximately 1,500 universities ranked by Quacquarelli Symonds (QS). The dataset includes numerous attributes, including a school's name, location, region, 2024 and 2025 rankings, size, focus, status, and numerous other miscellaneous rankings, as well as an overall score.

## Data Analysis Questions

1. How many of the top 100 universities are in each country?
2. What is the average overall score of the top 600 universities in each region?
3. Which universities that were ranked in 2024 have increased in rank in 2025, and by how much?
4. How do sustainability scores between public and private universities compare?
5. Which universities have a perfect academic reputation score?
6. How many unversities of each size are there?
7. Which unversities have the most cited research?
8. Which unversities have the highest international student ratio?

## Imports

In [1]:
import pandas as pd

## Load the data

In [2]:
df = pd.read_csv("QS World University Rankings 2025.csv", encoding="latin1")

## Inspect and Clean the Data

### Check For Null Values

The first step in our data cleaning is to determine which columns have null values, how many null values they have, and how to deal with these columns.

In [3]:
df.isnull().sum().sort_values(ascending=False)

Overall_Score                           902
International_Faculty_Rank              100
International_Faculty_Score             100
International_Students_Score             58
International_Students_Rank              58
STATUS                                   37
RANK_2024                                21
Sustainability_Score                     19
Sustainability_Rank                      19
International_Research_Network_Score      1
International_Research_Network_Rank       1
Institution_Name                          0
Employer_Reputation_Score                 0
Academic_Reputation_Rank                  0
Academic_Reputation_Score                 0
RES.                                      0
SIZE                                      0
FOCUS                                     0
Region                                    0
Location                                  0
RANK_2025                                 0
Citations_per_Faculty_Rank                0
Faculty_Student_Rank            

We will use two different methods to deal with columns with null values. For the "Overall_Score" column, we will only use this attribute to analyze the top 600 universities, as those are the schools that were given an overall score by QS. For all other columns, we will exclude rows with null values only for any queries involving the attribute in question. This will better maintain the integrity of the data than filling null values with the median or mean of the column, as carelessly filling values with summary statistics could misrepresent the outputs of our queries. Additionally, there are relatively few rows with null values for any given attribute in the grand scheme of the dataset, so the results of our queries will remain reliable.

### Clean the Data

In [4]:
# Clean column names
df.columns = df.columns.str.strip().str.lower()

# Clean and convert the rank columns
df["rank_2025"] = df["rank_2025"].astype(str).str.replace("=", "").str.extract(r"(\d+)").astype("Int64")
df["rank_2024"] = df["rank_2024"].astype(str).str.replace("=", "").str.extract(r"(\d+)").astype("Int64")

# Clean and convert the "sustainability_score" column
df["sustainability_score"] = pd.to_numeric(
    df["sustainability_score"]
    .astype(str)
    .str.replace("=", "", regex=False),
    errors="coerce"
)

# Convert the "overall_score" column to numeric
df["overall_score"] = pd.to_numeric(df["overall_score"], errors="coerce")

# Clean and convert "rank" columns with + signs
rank_cols = [col for col in df.columns if "rank" in col.lower()]

for col in rank_cols:
    df[col] = pd.to_numeric(
        df[col].astype(str).str.replace("+", "", regex=False), 
        errors="coerce"
    )

# Rename and fill values in the "status" column
df["status"] = df["status"].replace({
    "A": "Public",
    "B": "Private",
    "C": "Unknown"
}).fillna("Unknown")


To clean the data, we need to do a few things. First, we strip the column names of any white space and set them to lowercase to ensure easy access in future code. Then, we make sure that each value in the rank_2024 and rank_2025 columns is a single float value instead of a range like "601-610". After that, we clean the sustainability_score and any columns with "rank" in the name, removing any "+" or "=" signs in the cell values and ensuring the values are numeric. Finally, we improve the clarity of the status column, replacing values of "A", "B", and "C" with "Public", "Private", and "Unkown", respectively, as well as filling null values with "Unknown".

## Data Analysis

### 1. How many of the top 100 universities are in each country?

In [5]:
# Select only universities ranked within the top 100
top_100 = df[df["rank_2025"] <= 100]

# Count the number of top 100 universities in each country
top_100_by_country = top_100["location"].value_counts()

# Print the countries and their counts
print(top_100_by_country)

location
United States       25
United Kingdom      15
Australia            9
Hong Kong SAR        5
China (Mainland)     5
South Korea          5
Germany              5
France               4
Canada               4
Japan                4
Singapore            2
Switzerland          2
Netherlands          2
Sweden               2
Malaysia             1
New Zealand          1
Belgium              1
Taiwan               1
Argentina            1
Ireland              1
Brazil               1
Chile                1
Russia               1
Mexico               1
Denmark              1
Name: count, dtype: int64


### 2. What is the average overall score of the top 600 universities in each region?

In [6]:
# Select only universities ranked within the top 600
top_600 = df[df["rank_2025"] <= 600]

# Take the mean of the overall scores of the universities in each region
avg_score_by_region = top_600.groupby("region")["overall_score"].mean()

# Sort the countries by mean descending
avg_score_by_region = avg_score_by_region.sort_values(ascending=False)

# Print the regions and scores in descending order
print(avg_score_by_region)

region
Oceania     47.323077
Americas    47.178862
Europe      41.023887
Asia        38.607143
Africa      32.911111
Name: overall_score, dtype: float64


### 3. Which universities that were ranked in 2024 have increased in rank in 2025, and by how much?

In [7]:
# Filter out rows with missing 2024 rank data
ranked_both_years = df[df["rank_2024"].notna()]

# Create a new data frame including only rows where the rank has improved between years
improved = ranked_both_years[ranked_both_years["rank_2025"] < ranked_both_years["rank_2024"]].copy()

# Ensure rank 2024 is type int
improved["rank_2024"] = improved["rank_2024"].astype("Int64")

# Create a new row indicating how much a university's rank improved
improved["rank_change"] = improved["rank_2024"] - improved["rank_2025"]

# Print only relevant attributes of the new data frame
print(improved[["institution_name", "rank_2024", "rank_2025", "rank_change"]])

                                      institution_name  rank_2024  rank_2025  \
1                              Imperial College London          6          2   
9         California Institute of Technology (Caltech)         15         10   
10                          University of Pennsylvania         12         11   
12                         The University of Melbourne         14         13   
13                                   Peking University         17         14   
...                                                ...        ...        ...   
1323                           Universidad de Tarapaca       1401       1201   
1335  Universidad Nacional Autónoma de Honduras (UNAH)       1401       1201   
1341                      Universidad Nacional del Sur       1401       1201   
1351       Universidade Federal do Rio Grande Do Norte       1401       1201   
1368                          University of Kragujevac       1401       1201   

      rank_change  
1               4  

### 4. How do sustainability scores between public and private universities compare?

In [8]:
# Filter out rows with missing sustainability scores
sustainability_by_type = df[df["sustainability_score"].notna()]

# Group by "status" and calculate both mean and median
sustainability_stats = sustainability_by_type.groupby("status")["sustainability_score"].agg(["mean", "median"])

# Sort the university types by mean descending
sustainability_stats = sustainability_stats.sort_values(by="mean", ascending=False)

# Print the mean and median by unversity type
print(sustainability_stats)

              mean  median
status                    
Public   27.031665    9.50
Private  18.321429    2.95
Unknown   4.422989    1.20


### 5. Which universities have a perfect academic reputation score?

In [9]:
# Select only rows with an academic reputation score of 100 (a perfect score)
perfect_reputation = df[df["academic_reputation_score"] == 100]

# Print the name and score of each university with a perfect score
print(perfect_reputation[["rank_2025", "institution_name", "academic_reputation_score"]])

    rank_2025                              institution_name  \
0           1  Massachusetts Institute of Technology (MIT)    
2           3                          University of Oxford   
3           4                            Harvard University   
4           5                       University of Cambridge   
5           6                           Stanford University   
11         12      University of California, Berkeley (UCB)   
32         32                       The University of Tokyo   
41         42  University of California, Los Angeles (UCLA)   

    academic_reputation_score  
0                       100.0  
2                       100.0  
3                       100.0  
4                       100.0  
5                       100.0  
11                      100.0  
32                      100.0  
41                      100.0  


### 6. How many unversities of each size are there?

In [10]:
# Get the counts of each size university
size_counts = df["size"].value_counts()

# Reorder the data frame in size order descending
ordered_sizes = size_counts.reindex(["XL", "L", "M", "S"])

# Output the size counts
print(ordered_sizes)

size
XL    352
L     691
M     372
S      88
Name: count, dtype: int64


### 7. Which unversities have the most cited research?

In [11]:
# Sort the data frame by highest citation per faculty score
top_citations = df.sort_values(by="citations_per_faculty_score", ascending=False)

# Print the rank, name, and score of the first top 10 universities by citation per faculty score
print(top_citations[["rank_2025", "institution_name", "citations_per_faculty_score"]].head(10))

     rank_2025                                   institution_name  \
0            1       Massachusetts Institute of Technology (MIT)    
9           10       California Institute of Technology (Caltech)   
3            4                                 Harvard University   
61          62                       City University of Hong Kong   
358        359  Gwangju Institute of Science and Technology (G...   
382        383                                    Anna University   
402        403                 Amirkabir University of Technology   
21          22                               Princeton University   
182        183                       Hamad Bin Khalifa University   
325        326  Daegu Gyeongbuk Institute of Science and Techn...   

     citations_per_faculty_score  
0                          100.0  
9                          100.0  
3                          100.0  
61                         100.0  
358                        100.0  
382                        100

### 8. Which unversities have the highest international student ratio?

In [12]:
# Sort the data frame by highest international student ratio
top_international_students = df.sort_values(by="international_students_score", ascending=False)

# Print the rank, name, and score of the first top 10 universities by international student ratio
print(top_international_students[["rank_2025", "institution_name", "international_students_score"]].head(10))

      rank_2025                                   institution_name  \
25           26                                               EPFL   
17           18                           The University of Sydney   
1056       1001                     London Metropolitan University   
463         464         Macau University of Science and Technology   
51           50  The London School of Economics and Political S...   
802         801                        Al-Ahliyya Amman University   
748         741                          University of Westminster   
61           62                       City University of Hong Kong   
890         851                        University of Hertfordshire   
40           40                       The University of Queensland   

      international_students_score  
25                           100.0  
17                           100.0  
1056                         100.0  
463                          100.0  
51                           100.0  
802    

## Concluding Thoughts